# EventinatR usage

The following shows a simplified usage of the EventinatR objects.

In [ ]:
#!pwsh
Remove-Item -Path ~\.nuget\packages\eventinatr\ -Recurse
dotnet pack ..\..\EventinatR.sln --output C:\Windows\Temp\EventinatR

In [ ]:
#i "nuget:C:\Windows\Temp\EventinatR"
#r "nuget:EventinatR"
#r "nuget:System.Memory.Data"
#r "nuget:System.Linq.Async"

using System;
using System.Collections.Generic;
using System.Linq;
using EventinatR;
using EventinatR.InMemory;

# Domain model

We are modeling a simple group with members, with the ability to add and remove members and the events that change the state of the group.

In [ ]:
public record GroupMember(string Name);
public record GroupId(string Name);

public abstract record Event;

public abstract record GroupEvent : Event
{
    public record Created(GroupId Id) : GroupEvent;
    public record AddedMember(GroupId Id, GroupMember Member) : GroupEvent;
    public record RemovedMember(GroupId Id, GroupMember Member) : GroupEvent;
}

public record GroupState(GroupId Id, IEnumerable<GroupMember> Members);

public class Group
{
    public GroupId Id { get; private set; }
    public IEnumerable<GroupMember> Members => _members.AsEnumerable();
    public ICollection<GroupEvent> UncommittedEvents { get; } = new List<GroupEvent>();
    public GroupState State => new GroupState(Id, Members);

    private readonly List<GroupMember> _members = new();

    public Group(string name)
    {
        AddEvent(new GroupEvent.Created(new GroupId(name)));
    }

    public Group(IEnumerable<GroupEvent> events, GroupState state = null)
    {
        ArgumentNullException.ThrowIfNull(events);

        if (!events.Any() && state is null)
        {
            throw new ArgumentException("At least one event or state must be provided.");
        }

        if (state is not null)
        {
            Id = state.Id;
            _members.AddRange(state.Members);
        }

        foreach (var e in events)
        {
            ApplyEvent(e);
        }
    }

    private void AddEvent(GroupEvent e)
    {
        ApplyEvent(e);
        UncommittedEvents.Add(e);
    }

    private void ApplyEvent(GroupEvent e)
    {
        switch (e)
        {
            case GroupEvent.Created created:
                Apply(created);
                break;
            case GroupEvent.AddedMember addedMember:
                Apply(addedMember);
                break;
            case GroupEvent.RemovedMember removedMember:
                Apply(removedMember);
                break;
            default:
                throw new InvalidOperationException($"Unsupported event: {e.GetType().FullName}");
        }
    }

    private void Apply(GroupEvent.Created e)
        => Id = e.Id;

    public void AddMember(string name)
    {
        if (!_members.Any(x => x.Name == name))
        {
            var member = new GroupMember(name);
            var e = new GroupEvent.AddedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.AddedMember e)
        => _members.Add(e.Member);

    public void RemoveMember(string name)
    {
        var member = _members.FirstOrDefault(x => x.Name == name);

        if (member is not null)
        {
            var e = new GroupEvent.RemovedMember(Id, member);
            AddEvent(e);
        }
    }

    private void Apply(GroupEvent.RemovedMember e)
        => _members.Remove(e.Member);
}

public class GroupRepository
{
    private readonly EventStore _store;

    public GroupRepository(EventStore store)
        => _store = store;

    public Task<EventStream> GetStreamAsync(GroupId id)
        => _store.GetStreamAsync($"{nameof(Group)}:{id.Name}".ToLowerInvariant());

    public Task<Group?> GetAsync(GroupId id)
        => GetAsync(id.Name);

    public async Task<Group?> GetAsync(string id)
    {
        var stream = await GetStreamAsync(new GroupId(id));
        var snapshot = await stream.ReadSnapshotAsync<GroupState>();
        var state = snapshot.State;
        var events = await snapshot.ReadAsync().Select(e =>
            {
                if (!e.TryConvert<GroupEvent>(out var groupEvent))
                {
                    throw new InvalidOperationException($"The event stream contains data that is not supported: {e.Data}");
                }

                return groupEvent;
            }).ToListAsync();

        return new Group(events, state);
    }

    public async Task<EventStreamVersion> SaveAsync(Group group)
    {
        ArgumentNullException.ThrowIfNull(group);

        if (group.UncommittedEvents.Any())
        {
            var stream = await GetStreamAsync(group.Id);
            var version = await stream.AppendAsync(group.UncommittedEvents, group.State);
            group.UncommittedEvents.Clear();
            return version;
        }

        return EventStreamVersion.None;
    }
}

# Saving events to the stream

1. Initialize an in-memory event store
2. Create the aggregate root and perform some commands to generate events
3. Get a stream for the group from the event store
4. Save the events to the stream

In [ ]:
EventStore store = new InMemoryEventStore();
var repository = new GroupRepository(store);

var group = new Group("Final Fantasty Games");

group.AddMember("Final Fantasy I");
group.AddMember("Final Fantasy II");
group.AddMember("Final Fantasy III");
group.AddMember("Final Fantasy IV");
group.AddMember("Final Fantasy V");
group.AddMember("Final Fantasy VI");
group.AddMember("Final Fantasy VII");
group.AddMember("Final Fantasy VIII");
group.AddMember("Final Fantasy IX");
group.AddMember("Final Fantasy X");
group.AddMember("Final Fantasy XI");
group.AddMember("Final Fantasy XII");
group.AddMember("Final Fantasy XIII");
group.AddMember("Final Fantasy XIV");
group.AddMember("Final Fantasy XV");

await repository.SaveAsync(group);

display(group);

# Reading events from a stream

The aggregate root was designed to be able to read from a stream and get back to current state, the output should be exactly what was rendered in the previous step.

In [ ]:
group = await repository.GetAsync(group.Id);
display(group);

# Generate some more events

Final Fantasy XIV was never really a game until A Realm Reborn was released.

In [ ]:
group.RemoveMember("Final Fantasy XIV");
group.AddMember("Final Fantasty XIV - A Realm Reborn");
await repository.SaveAsync(group);

display(group);

# Show the events from a stream

You can see each version, the type of event, and the event data.

In [ ]:
var stream = await repository.GetStreamAsync(group.Id);
var events = await stream.ReadAsync().ToListAsync();
display(events);